# Эксперимент по использованию метода кросс-доменной адаптации PUL на моделях распознавания ключевых точек от проекта MMPose



## Установка зависимостей

In [ ]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/usr/local/bin/python


In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 578.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.0+cu121
    Uninstalling torc

In [ ]:
!pip install -U openmim
!mim install mmengine
!mim install "mmcv==2.1.0"
!mim install "mmdet==3.3.0"

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 23.3 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/mmcv-2.1.0-cp310-cp310-manylinux1_x86_64.whl (94.1 MB)
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
%pip install -r requirements.txt
%pip install -v -e .

Cloning into 'mmpose'...
remote: Enumerating objects: 31111, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 31111 (delta 89), reused 159 (delta 67), pack-reused 30895
Receiving objects: 100% (31111/31111), 53.40 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (21914/21914), done.
/content/mmpose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 2.1.0+cu121 True
torchvision version: 0.16.0+cu121
mmpose version: 1.3.1
cuda version: 12.1
compiler information: GCC 9.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Эксперимент

### Общие методы для парсинга результата и высчитывания метрик

In [ ]:
import numpy as np
from mmengine.structures.instance_data import InstanceData

def convert_visability_to_coco(keypoints: np.ndarray, visability_threshold: float) -> np.ndarray:
    # Set visability in COCO format:
    #  - v=0: not labeled (in which case x=y=0)
    #  - v=1: labeled but not visible (visability less then threshold)
    #  - v=2: labeled and visible (visability more or equal then threshold)
    for keypoint in keypoints:
        if (keypoint[2] > visability_threshold):
            keypoint[2] = 2
        else:
            keypoint[2] = 1

    return keypoints


def instanсe_data_to_keypoints(points, visability):
    return np.column_stack((points, visability))


def parse_inference_results(pred_instances: InstanceData) -> tuple[np.ndarray, dict]:
    bsi = np.argmax(pred_instances.keypoint_scores.mean(
        axis=1).astype(float))  # best_score_index = bsi
    scores = pred_instances.keypoint_scores[bsi]
    keypoints = instanсe_data_to_keypoints(
        pred_instances.keypoints[bsi], pred_instances.keypoints_visible[bsi])
    keypoints = convert_visability_to_coco(keypoints, 0.5)

    x_max = keypoints[:, 0].max()
    x_min = keypoints[:, 0].min()
    y_max = keypoints[:, 1].max()
    y_min = keypoints[:, 1].min()
    width = x_max - x_min
    height = y_max - y_min

    annotation = {
        'keypoints': keypoints.flatten().astype(int).tolist(),
        'num_keypoints': len(keypoints),
        'iscrowd': 0,
        'bbox': np.array([round(x_min, 2), round(y_min, 2),
                 round(width, 2), round(height, 2)]).astype(float).tolist(),
        'area': float(round(width*height, 3)),
    }

    return (scores, annotation)

In [ ]:
def calc_distances(pred_kpt, real_kpt):
    return np.sqrt(((pred_kpt[:, :2] - real_kpt[:, :2]) ** 2).sum(axis=1))


def calc_pck(pred_kpt, annotation, threshold=0.05):
    real_kpt = np.array(annotation['keypoints']).reshape((-1, 3))
    height = annotation['bbox'][3]
    dists = calc_distances(pred_kpt, real_kpt)
    return np.sum(dists < threshold * height) / annotation['num_keypoints']


coco_sigmas = np.array([
    0.026, 0.025, 0.025, 0.035, 0.035, 0.079, 0.079, 0.072, 0.072, 0.062,
    0.062, 0.107, 0.107, 0.087, 0.087, 0.089, 0.089
])


def calc_oks(pred_kpt, annotation, sigmas=coco_sigmas):
    real_kpt = np.array(annotation['keypoints']).reshape((-1, 3))
    area = annotation['area']

    dists = calc_distances(pred_kpt, real_kpt)
    visible_mask = real_kpt[:, 2] > 0
    return np.sum(np.exp(- dists**2 / 2 / area / (2*sigmas)**2)[visible_mask]) / visible_mask.sum()


def calc_ap(oks_scores, oks_threshold=0.5):
    tp = oks_scores >= oks_threshold
    fp = oks_scores < oks_threshold

    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)

    precision = cum_tp / (cum_tp + cum_fp)
    recall = cum_tp / len(oks_scores)

    # Вычисление площади под кривой Precision-Recall
    ap = np.trapz(precision, recall)
    return ap


### Тестирование предобученной модели на тестовых данных

In [ ]:
from mmpose.apis import inference_topdown, init_model

model_cfg = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py'
ckpt = 'https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth'

device = 'cuda'

# init model
model = init_model(model_cfg, ckpt, device=device)

In [ ]:
import json

test_data_folder = "../drive/MyDrive/diplom_dataset/train_dataset/data/test/"
test_annotations = {}

with open(test_data_folder + "annotations.json", "r") as file:
    test_annotations = json.load(file)

images = test_annotations['images']
annotations = test_annotations['annotations']
img_id_to_ann = {ann['image_id']: ann for ann in annotations}

In [ ]:
from tqdm import tqdm
import time

pck_005_s = []
pck_01_s = []
pck_025_s = []
okss = []
pred_time = []
for image in tqdm(images):
    start = time.time()
    inference_res = inference_topdown(model, test_data_folder + 'images/' + image['file_name'])
    end = time.time()
    (_, ann) = parse_inference_results(inference_res[0].pred_instances)
    pred_keypoints = np.array(ann['keypoints']).reshape((-1,3))

    pck_005_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.05))
    pck_01_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.1))
    pck_025_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.25))
    okss.append(calc_oks(pred_keypoints, img_id_to_ann[image['id']]))
    pred_time.append((end - start) * 1000)

In [ ]:
print(f"AP@50 = {round(calc_ap(np.array(okss), 0.5), 3)}")
print(f"AP@75 = {round(calc_ap(np.array(okss), 0.75), 3)}")
print(f"mAP = {np.array([calc_ap(np.array(okss), th) for th in np.arange(0.5, 1.0, 0.05)]).mean().round(3)}")
print()
print(f"mean PCK@0.05 = {np.mean(pck_005_s).round(3)}")
print(f"mean PCK@0.1  = {np.mean(pck_01_s).round(3)}")
print(f"mean PCK@0.25 = {np.mean(pck_025_s).round(3)}")
print()
print(f"Average time to predict keypoints {np.array(pred_time).mean().round(3)} ms")

### Методы для генерации псевдо лейблов

In [ ]:
from PIL import Image
from tqdm import tqdm
import os


def split_indexes(sample_size, test_data_part=0.2):
    print(f"Split indexes with test_size: {test_data_part}")
    test_size = int(sample_size * test_data_part)

    indexes = np.random.permutation(sample_size)

    test_indexes = indexes[:test_size]
    train_indexes = indexes[test_size:]

    return (train_indexes, test_indexes)


def generate_pseudo_labels(model, path_to_images, confidence_threshold=0.8):
    image_filenames = os.listdir(path_to_images)
    annotations = []
    image_ind = 0
    images = []
    mean_scores = []

    print(f"Generate pseudo labels with confidence threshold: {confidence_threshold}")
    for image_filename in tqdm(image_filenames):
        results = inference_topdown(model, path_to_images + image_filename)
        (scores, annotation) = parse_inference_results(
            results[0].pred_instances)
        scores = scores / scores.max()
        mean_scores.append(scores.mean())

        if (scores.mean() > confidence_threshold):
            tmp_img = Image.open(path_to_images + image_filename)
            (width, height) = tmp_img.size
            img_json = {
                'file_name': image_filename,
                'height': height,
                'width': width,
                'id': image_ind
            }
            images.append(img_json)

            annotation['image_id'] = image_ind
            annotation['category_id'] = 1
            annotation['id'] = image_ind
            annotations.append(annotation)

            image_ind += 1

    for th in np.arange(confidence_threshold - 0.1,  1.04, 0.01).round(2):
        print(f"With th={th} was generate {(np.array(mean_scores) > th).sum()} labels")

    return (images, annotations)


def generate_pseudo_coco_annotations(model, path_to_images, confidence_threshold=0.8, test_data_part=0.2):
    (images, pseudo_annotations) = generate_pseudo_labels(
        model, path_to_images, confidence_threshold)

    sample_size = 0
    if len(images) != len(pseudo_annotations):
        raise IndexError("Size of images and annotations must be equal!")
    else:
        sample_size = len(images)

    print(f"Was generate {sample_size} pseudo-labels")
    (train_indexes, test_indexes) = split_indexes(sample_size, test_data_part)

    print("Split pseudo-labels:")
    print(f"  train_size = {len(train_indexes)}")
    print(f"  test_size = {len(test_indexes)}")

    np_images = np.array(images)
    np_annotations = np.array(pseudo_annotations)

    train_json = {
        "images": np_images[train_indexes].tolist(),
        "annotations": np_annotations[train_indexes].tolist(),
        "categories": [{"supercategory": "person", "id": 1, "name": "person"}]
    }

    test_json = {
        "images": np_images[test_indexes].tolist(),
        "annotations": np_annotations[test_indexes].tolist(),
        "categories": [{"supercategory": "person", "id": 1, "name": "person"}]
    }

    return (train_json, test_json)

### Итеративное обучение модели на данных

#### Генерация псевдолейблов

In [ ]:
# Install this utily if work with ViTPose
# !mim install 'mmpretrain>=1.0.0'

In [ ]:
from mmpose.apis import inference_topdown, init_model

model_cfg = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-base_8xb64-210e_coco-256x192.py'
ckpt = '../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/vitpose/iter_2.pth'

device = 'cuda'

# init model
model = init_model(model_cfg, ckpt, device=device)

Loads checkpoint by local backend from path: ../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/vitpose/iter_2.pth


In [ ]:
import json

images_folder = "../drive/MyDrive/diplom_dataset/train_dataset/data/images/"

(train_pseudo_labels, val_pseudo_labels) = generate_pseudo_coco_annotations(
    model, images_folder, confidence_threshold=0.8, test_data_part=0.15)

Generate pseudo labels with confidence threshold: 0.7


100%|██████████| 2232/2232 [03:59<00:00,  9.33it/s]

With th=0.6 was generate 2232 labels
With th=0.61 was generate 2232 labels
With th=0.62 was generate 2232 labels
With th=0.63 was generate 2232 labels
With th=0.64 was generate 2232 labels
With th=0.65 was generate 2232 labels
With th=0.66 was generate 2232 labels
With th=0.67 was generate 2232 labels
With th=0.68 was generate 2232 labels
With th=0.69 was generate 2231 labels
With th=0.7 was generate 2231 labels
With th=0.71 was generate 2229 labels
With th=0.72 was generate 2225 labels
With th=0.73 was generate 2221 labels
With th=0.74 was generate 2220 labels
With th=0.75 was generate 2211 labels
With th=0.76 was generate 2197 labels
With th=0.77 was generate 2182 labels
With th=0.78 was generate 2138 labels
With th=0.79 was generate 2072 labels
With th=0.8 was generate 2001 labels
With th=0.81 was generate 1877 labels
With th=0.82 was generate 1717 labels
With th=0.83 was generate 1504 labels
With th=0.84 was generate 1225 labels
With th=0.85 was generate 952 labels
With th=0.86 was

In [ ]:
with open("../drive/MyDrive/diplom_dataset/train_dataset/data/annotations/pul_mean_score/vitpose/iter_2_train.json", 'w') as file:
    json.dump(train_pseudo_labels, file)
with open("../drive/MyDrive/diplom_dataset/train_dataset/data/annotations/pul_mean_score/vitpose/iter_2_val.json", 'w') as file:
    json.dump(val_pseudo_labels, file)

#### Обучение на псевдолейблах

In [ ]:
from mmengine import Config

cfg = Config.fromfile(
    'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-base_8xb64-210e_coco-256x192.py'
)

# set pretrained ckpt
cfg.resume = True
cfg.load_from = '../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/vitpose/iter_1.pth'

# set basic configs
cfg.data_root = '../drive/MyDrive/diplom_dataset/train_dataset/data/'
cfg.work_dir = '../drive/MyDrive/diplom_dataset/train_dataset/work_dirs/pul_mean_score/vitpose/iter_2/'
cfg.randomness = dict(seed=0)

# set log interval
cfg.train_cfg.val_interval = 5

# set num of epoch
cfg.train_cfg.max_epochs = 60

# set optimizer
cfg.optim_wrapper = dict(optimizer=dict(
    type='Adam',
    lr=5e-4,
))

# set learning rate policy
cfg.param_scheduler = [
    dict(
        type='LinearLR', begin=0, end=10, start_factor=0.001,
        by_epoch=False),  # warm-up
    dict(
        type='MultiStepLR',
        begin=0,
        end=cfg.train_cfg.max_epochs,
        milestones=[17, 35],
        gamma=0.1,
        by_epoch=True)
]


# set batch size
cfg.train_dataloader.batch_size = 16
cfg.val_dataloader.batch_size = 16
cfg.test_dataloader.batch_size = 16

cfg.train_dataloader.dataset.ann_file = 'annotations/pul_mean_score/vitpose/iter_2_train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img='images/')

cfg.val_dataloader.dataset.bbox_file = None
cfg.val_dataloader.dataset.ann_file = 'annotations/pul_mean_score/vitpose/iter_2_val.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img='images/')

cfg.test_dataloader.dataset.bbox_file = None
cfg.test_dataloader.dataset.ann_file = 'annotations/pul_mean_score/vitpose/iter_2_val.json'
cfg.test_dataloader.dataset.data_root = cfg.data_root
cfg.test_dataloader.dataset.data_prefix = dict(img='images/')

# set evaluator
cfg.val_evaluator = dict(type='PCKAccuracy')
cfg.test_evaluator = cfg.val_evaluator

cfg.default_hooks.checkpoint.save_best = 'PCK'
cfg.default_hooks.checkpoint.max_keep_ckpts = 1

# print(cfg.pretty_text)

In [ ]:
from mmengine.runner import Runner

# set preprocess configs to model
cfg.model.setdefault('data_preprocessor', cfg.get('preprocess_cfg', {}))

# build the runner from config
runner = Runner.from_cfg(cfg)

# start training
runner.train()

06/09 20:12:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-genc

/usr/local/lib/python3.10/dist-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmpose.visualization.local_visualizer.PoseLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


06/09 20:12:45 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
06/09 20:12:46 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
06/09 20:13:28 - mmengine - WARNING - The prefix is not set in metric class PCKAccuracy.
06/09 20:13:28 - mmengine - INFO - load model from: https://download.openmmlab.com/mmpose/v1/pretrained_models/mae_pretrain_vit_base_20230913.pth
06/09 20:13:28 - mmengine - INFO - Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/pretrained_models/mae_pretrain_vit_base_20230913.pth
06/09 20:13:29 - mmengine - INFO - Resize the pos_embed shape from torch.Size([1, 196, 768]) to torch.Size([1, 192, 768]).
06/09 20:13:29 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: cls_token

Loads checkpoint by local backend from path: ../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/vitpose/iter_1.pth
06/09 20:13:31 - mmengine - INFO - Load checkpoint from ../drive/MyDrive/diplom_dataset/train_dataset/experime

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


06/09 20:14:07 - mmengine - INFO - Epoch(train) [42][ 50/115]  lr: 5.000000e-05  eta: -1 day, 22:53:14  time: 0.724222  data_time: 0.093824  memory: 4510  loss: 0.001121  loss_kpt: 0.001121  acc_pose: 0.806082
06/09 20:14:41 - mmengine - INFO - Epoch(train) [42][100/115]  lr: 5.000000e-05  eta: -1 day, 22:54:46  time: 0.678278  data_time: 0.058092  memory: 4510  loss: 0.000898  loss_kpt: 0.000898  acc_pose: 0.829264
06/09 20:14:52 - mmengine - INFO - Exp name: td-hm_ViTPose-base_8xb64-210e_coco-256x192_20240609_201243
06/09 20:15:32 - mmengine - INFO - Epoch(train) [43][ 50/115]  lr: 5.000000e-05  eta: -1 day, 22:51:12  time: 0.800567  data_time: 0.134560  memory: 4510  loss: 0.000854  loss_kpt: 0.000854  acc_pose: 0.773602
06/09 20:16:06 - mmengine - INFO - Epoch(train) [43][100/115]  lr: 5.000000e-05  eta: -1 day, 22:51:29  time: 0.690955  data_time: 0.067639  memory: 4510  loss: 0.000843  loss_kpt: 0.000843  acc_pose: 0.846488
06/09 20:16:17 - mmengine - INFO - Exp name: td-hm_ViTPo

TopdownPoseEstimator(
  (data_preprocessor): PoseDataPreprocessor()
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=(2, 2))
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (ln1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MultiheadAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (out_drop): DropPath()
          (gamma1): Identity()
        )
        (ln2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (ffn): FFN(
          (layers): Sequential(
            (0): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU(approximate='none')
         

In [ ]:
# save checkpoint
runner.save_checkpoint("../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/vitpose/", 'iter_2.pth')

#### Тестирование обученной модели на тестовых данных

In [ ]:
from mmpose.apis import inference_topdown, init_model

model_cfg = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py'
ckpt = "../drive/MyDrive/diplom_dataset/train_dataset/experiment/pul_mean_score/hrnet/iter_1.pth"

device = 'cuda'

# init model
model = init_model(model_cfg, ckpt, device=device)

In [ ]:
import json

test_data_folder = "../drive/MyDrive/diplom_dataset/train_dataset/data/test/"
test_annotations = {}

with open(test_data_folder + "annotations.json", "r") as file:
    test_annotations = json.load(file)

images = test_annotations['images']
annotations = test_annotations['annotations']
img_id_to_ann = {ann['image_id']: ann for ann in annotations}

In [ ]:
from tqdm import tqdm
import time

pck_005_s = []
pck_01_s = []
pck_025_s = []
okss = []
pred_time = []
for image in tqdm(images):
    start = time.time()
    inference_res = inference_topdown(model, test_data_folder + 'images/' + image['file_name'])
    end = time.time()
    (_, ann) = parse_inference_results(inference_res[0].pred_instances)
    pred_keypoints = np.array(ann['keypoints']).reshape((-1,3))

    pck_005_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.05))
    pck_01_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.1))
    pck_025_s.append(calc_pck(pred_keypoints, img_id_to_ann[image['id']], threshold=0.25))
    okss.append(calc_oks(pred_keypoints, img_id_to_ann[image['id']]))
    pred_time.append((end - start) * 1000)

In [ ]:
print(f"AP@50 = {round(calc_ap(np.array(okss), 0.5), 3)}")
print(f"AP@75 = {round(calc_ap(np.array(okss), 0.75), 3)}")
print(f"mAP = {np.array([calc_ap(np.array(okss), th) for th in np.arange(0.5, 1.0, 0.05)]).mean().round(3)}")
print()
print(f"mean PCK@0.05 = {np.mean(pck_005_s).round(3)}")
print(f"mean PCK@0.1  = {np.mean(pck_01_s).round(3)}")
print(f"mean PCK@0.25 = {np.mean(pck_025_s).round(3)}")
print()
print(f"Average time to predict keypoints {np.array(pred_time).mean().round(3)} ms")

# DRAFT